In [1]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator
import keras_video.utils

In [2]:
#Global Params
SIZE = (112, 112)
CHANNELS = 3
NBFRAME = 5
BS = 8
EPOCHS=50
CONVSHAPE=SIZE + (CHANNELS,)
INSHAPE=(NBFRAME,) + CONVSHAPE # (5, 112, 112, 3)

In [3]:
classes = [i.split(os.path.sep)[1] for i in glob.glob('../../data/train/*')]
classes.sort()
print(classes)

['FAKE', 'REAL']


In [4]:
#pattern to get videos and classes
glob_pattern = '../../data/train/{classname}/*.mp4'

#Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=False
)

valid = train.get_validation_generator()

class FAKE, validation count: 115, train count: 236
class REAL, validation count: 34, train count: 71
Total data: 2 classes for 307 files for train
Total data: 2 classes for 149 files for validation


In [5]:
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
def build_convnet(shape=CONVSHAPE):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

In [6]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
def action_model(shape=INSHAPE, nbout=2):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [7]:
#Compile model
model = action_model(INSHAPE, len(classes))
optimizer = keras.optimizers.Adam(0.001)
print(model.summary())
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 5, 512)            4689216   
_________________________________________________________________
gru (GRU)                    (None, 64)                110976    
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [ ]:
#Train model on dataset
callbacks = [keras.callbacks.ReduceLROnPlateau(verbose=1),]
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
#Export model
import h5py
model.save('../output_models/rc_base.h5')